In [ ]:
from core.board import Board
from analysis.msm_builder import get_msm_graph
from solver.solver import solve

In [ ]:
print('\a')

In [ ]:
r = 16
c = 30
n = 99
start = (0,0)
seed = 750854766      
preset = False
profile = False
B_t = Board.random_complete(r,c,n,exclude=[start],seed=seed)
assert B_t[start].M == 0
B_k = Board(r,c,n)
if not preset:
    B_k.set_clear(*start)
    B_k.project_from(B_t)
else:
    B_k.set_clear(0,0,1)
    B_k.set_clear(1,0,1)
    B_k.set_clear(2,0,1)
    B_k.set_clear(2,1,3)
    B_k.set_clear(0,2,2)
if preset: print(B_k)
elif seed is not None: print(B_t)

In [ ]:
from analysis.msm_analysis import find_solutions, seperate_connected_msm_components
from functools import reduce
if preset: 
    B_o, msm, c, f = get_msm_graph(B_k, order=2, verbose=0)
    print(B_o, msm, c, f)
    msms = seperate_connected_msm_components(msm)
    print(find_solutions(msms[0], verbose=0))

In [ ]:
import cProfile
if not preset and seed is not None:
    if profile:
        cProfile.run('print(solve(B_k, B_t, verbose=2)[0])')
    else:
        print(solve(B_k, B_t, verbose=3)[0])

In [ ]:
import random
if seed is not None or preset: assert False
start = (0,0)
iterations = 100
wins = 0
for i in range(iterations):
    while True:
        seed = random.randint(0, 2**32)
        B_t = Board.random_complete(r,c,n,exclude=[start], seed=seed)
        if B_t[start].M == 0: break
    B_k = Board(r,c,n)
    B_k.set_clear(*start)
    B_k.project_from(B_t)
    print(seed, end=' ')
    win, steps = solve(B_k, B_t, verbose=0)
    if win: wins += 1
seed = None
wins / iterations